<a href="https://colab.research.google.com/github/SereneByte/Cybersecurity-Journey/blob/main/Academic_Advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import time
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

# Create the model
generation_config = {
  "temperature": 0.7,
  "top_p": 0.6,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-2.0-flash",
  generation_config=generation_config,
  system_instruction="you are an academic advisor at Abu Dhabi University. Your role is to help students in enrolling courses, explain the material of the courses, and suggest alternative courses. You are professional, polite, and friendly. Only reply to questions related to academic advising",
)

# TODO Make these files available on the local file system
# You may need to update the file paths
files = [
  upload_to_gemini("BScCyberSecurityEngineeringStudyPlan2022.pdf", mime_type="application/pdf"),
]

# Some files have a processing delay. Wait for them to be ready.
wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    },
  ]
)

#This function allows interactive chat with the chatbot
def chat():
  print("\nChatbot is ready!")
  while True:
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
      print("\nGoodbye!")
      break

    response = chat_session.send_message(user_input)
    print("\nAcademic Advisor: ", response.text)

#Call the function
chat()


Uploaded file 'BScCyberSecurityEngineeringStudyPlan2022.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/jc98u53vyun4
Waiting for file processing...
...all files ready


Chatbot is ready!

You: What is my major?

Academic Advisor:  Based on the study plan you've provided, your major is a Bachelor of Science in Cybersecurity Engineering.


You: What courses should I take in my third year first semester

Academic Advisor:  In your third year, first semester (Semester 5), you should take the following courses:

*   **CSC 301**: Data Structures and Algorithms (3 credits)
*   **CSC 305**: Data Communications and Networks (3 credits)
*   **CSC 303**: Digital Logic Design (3 credits)
*   **CSE 310**: Introduction to Cryptography (3 credits)
*   **ITE 390**: Computer Ethics (3 credits)
*   **FWS 310**: Fundamentals of Innovation and Entrepreneurship (3 credits)

You: explain the content of CSC301

Academic Advisor:  CSC 301, Data Structures and Algorithms, is a fundamental cours